# LASSO (coordinate descent)

Trong notebook này, chúng ta sẽ triển khai bộ giải LASSO qua coordinate descent. Chúng ta sẽ:
* Viết một hàm chuẩn hóa các đặc trưng
* Triển khai coordinate descent cho LASSO
* Khám phán tác động của L1 penalty

## Như thường lệ

In [54]:
import sklearn
import pandas

## Load dữ liệu doanh số bán nhà

Tập dữ liệu doanh số bán nhà ở quận King, Seatle, WA. Nghe quen chứ?

In [55]:
full_data = pandas.read_csv("kc_house_data.csv", index_col=0)
# Trong tập dữ liệu, 'floors' được xác định là kiểu string,
# nên chúng ta sẽ chuyển đổi chúng thành int trước khi sử dụng dưới đây
full_data['floors'] = full_data['floors'].astype(int) 

Nếu muốn thực hiện bất kỳ "feature engineering" nào như tạo các đặc trưng mới hoặc điều chỉnh đặc trưng sẵn có, chúng ta có thể sửa DataFrame của pandas như trong lab trước (Lab 2). Tuy nhiên, với notebook này, chúng ta sẽ làm việc với các đặc trưng có sẵn.

## Import các hàm hữu ích từ notebook trước

Như trong Exercise 1, chúng ta chuyển đổi DataFrame thành ma trận Numpy 2D. Copy và paste `get_numpy_data()` từ exercise đó.

In [56]:
import numpy as np # điều này cho phép gọi numpy as np 

In [57]:
def get_numpy_data(data, features_title, output_title):
    if('constant' not in data):
        data['constant'] = 1 # đây là cách thêm cột constant. Chỉ thực hiện khi cần 
    # thêm cột 'constant' vào trước list các đặc trưng để chúng ta có thể trích xuất cùng với những thứ khác:
    features_title = ['constant'] + features_title # đây là cách kết hợp 2 list
    # chia dữ liệu thành sub-DataFrame chứa các đặc trưng đã chỉ định (gồm constant)
    # gọi nó là features_columns.
    features_columns = data[features_title]
    # dòng tiếp theo sẽ trích xuất ma trận numpy từ biến features_columns:
    feature_matrix = features_columns.values
    # truy xuất dữ liệu được liên kết với đầu ra trong pandas Series
    # gọi nó là output_column
    output_column = data[output_title]
    # tiếp theo sẽ chuyển đổi Series đã nhắc thành một mảng numpy
    output_array = output_column.values
    return(feature_matrix, output_array)

Cũng copy và paste cả hàm `predict_output()` để tính các dự đoán cho toàn bộ ma trận đặc trưng với ma trận và trọng số đã cho:

In [58]:
def predict_output(feature_matrix, weights):
    # giả sử ma trận feature_matrix chứa các đặc trưng ở dạng các cột và trọng số là mảng numpy tương ứng
    # tạo vectơ dự đoán sử dụng np.dot()
    predictions = np.dot(feature_matrix, weights)
    return predictions

## Chuẩn hóa các đặc trưng
Trong tập dữ liệu giá nhà, các đặc trưng thay đổi khá nhiều về độ lớn: ví dụ, `sqft_living` rất lớn so với `bedrooms`. Do đó, trọng số cho `sqft_living` sẽ nhỏ hơn rất nhiều so với trọng số cho `bedrooms`. Điều này khó giải quyết vì các trọng số "nhỏ" bị giảm đầu tiên khi `l1_penalty` tăng. 

Để công bằng với tất cả các đặc trưng, chúng ta cần **chuẩn hóa đặc trưng** như đã thảo luận trong các bài giảng: chia mỗi đặc trưng cho chuẩn 2 của nó để đặc trưng đã biến đổi có chuẩn 1.

Hãy xem chúng ta có thể thực hiện chuẩn hóa này dễ dàng thế nào với Numpy: trước tiên chúng ta sẽ xem xét một ma trận nhỏ.

In [59]:
X = np.array([[3.,5.,8.],[4.,12.,15.]])
print(X)

[[ 3.  5.  8.]
 [ 4. 12. 15.]]


Numpy cung cấp cách viết tắt để tính toán chuẩn 2 của mỗi cột:

In [60]:
norms = np.linalg.norm(X, axis=0) # cho [norm(X[:,0]), norm(X[:,1]), norm(X[:,2])]
print(norms)

[ 5. 13. 17.]


Để chuẩn hóa, hãy áp dụng phép chia element-wise (thực hiện trên các phần tử tương ứng):

In [61]:
print(X / norms) # cho [X[:,0]/norm(X[:,0]), X[:,1]/norm(X[:,1]), X[:,2]/norm(X[:,2])]

[[0.6        0.38461538 0.47058824]
 [0.8        0.92307692 0.88235294]]


Với viết tắt mà chúng ta vừa đề cập, hãy viết một hàm ngắn là `normalize_features(feature_matrix)`, hàm này chuẩn hóa các cột của ma trận đặc trưng đã cho. Hàm phải sẽ về một cặp `(normalized_features, norms)`, trong đó mục thứ hai chứa chuẩn của các đặc trưng ban đầu. Như đã thảo luận trong các bài giảng, chúng ta sẽ sử dụng các chuẩn này để chuẩn hóa dữ liệu kiểm tra theo cách mà chúng ta chuẩn hóa dữ liệu huấn luyện.

In [62]:
def normalize_features(feature_matrix):
    norms = np.linalg.norm(feature_matrix, axis=0)
    features = feature_matrix/norms
    return features, norms

Để kiểm tra đạo hàm, chạy cell sau:

In [63]:
features, norms = normalize_features(np.array([[3.,6.,9.],[4.,8.,12.]]))
print(features)
# sẽ in ra
# [[ 0.6  0.6  0.6]
#  [ 0.8  0.8  0.8]]
print(norms)
# sẽ in ra
# [5.  10.  15.]

[[0.6 0.6 0.6]
 [0.8 0.8 0.8]]
[ 5. 10. 15.]


## Triển khai Coordinate Descent với các đặc trưng được chuẩn hóa

Chúng ta tìm cách thu được một tập hợp trọng số thưa thớt bằng cách giảm thiểu hàm chi phí LASSO
<!-- ``` SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|). ``` -->
$cost = \sum (prediction - output)^2 + \lambda * \sum_{i \neq 0} |w_i|$

(Theo quy ước, chúng ta không bao hàm $w_0$ (độ chệch) trong phần tử L1 penalty. Chúng ta không bao giờ đẩy intercept thành 0.)

Dấu giá trị tuyệt đối làm cho hàm chi phí không thể phân biệt được, do đó gradient descent đơn giản không khả thi (bạn sẽ cần triển khai phương pháp subgradient descent). Thay vào đó, chúng ta sẽ sử dụng **coordinate descent**: ở mỗi lần lặp, chúng ta sẽ cố định tất cả các trọng số ngoại trừ trọng số `i` và tìm giá trị trọng số` i` thu nhỏ mục tiêu, tức là tìm: <br>
<!-- ``` argmin_{w[i]} [ SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|) ] ``` -->
$argmin_{w_i}(cost_i) = \sum (prediction - output)^2 + \lambda * \sum_{i \neq 0} |w_i| )$

trong đó tất cả các trọng số khác $w_i$(`w[i]`) được coi là không đổi. Chúng ta sẽ tối ưu hóa $w_i$ lần lượt, tuần hoàn nhiều lần qua các trọng số.
  1. Chọn tọa độ `i`
  2. Tính $w_i$ giảm thiểu hàm chi phí $cost = (\sum prediction - output) + \lambda * \sum_{i \neq 0} |w_i|$
  3. Lặp lại bước 1 và 2 cho tất cả các tọa độ nhiều lần. 

Với notebook này, chúng ta sử dụng **coordinate descent theo chu kỳ với các đặc trưng được chuẩn hóa**, trong đó chúng ta tuần hoàn qua các tọa độ theo thứ tự từ 0 đến (d-1) và giả sử các đặc trưng đã được chuẩn hóa như đã thảo luận ở trên. Công thức để tối ưu hóa từng tọa độ như sau:
<!-- ```
       ┌ (ro[i] + lambda/2)     if ro[i] < -lambda/2
w[i] = ├ 0                      if -lambda/2 <= ro[i] <= lambda/2
       └ (ro[i] - lambda/2)     if ro[i] > lambda/2
``` -->
$w_i = \left\{
\begin{array}{ll}
      \rho_i + \lambda / 2 & \rho_i < -\lambda/2 \\
      0 & -\lambda/2 \leq \rho_i \leq \lambda/2  \\
      \rho_i - \lambda / 2 & \rho_i > \lambda/2 \\
\end{array} 
\right. $

trong đó $\rho_i$(`ro[i]`) được xác định như sau:
<!-- ```ro[i] = SUM[ [feature_i]*(output - prediction + w[i]*[feature_i]) ]. ``` -->
$\rho_i = \sum feature_i * (output - prediction + w_i*feature_i)$

Lưu ý là chúng ta không điều chuẩn trọng số của đặc trưng không đổi (intercept|độ chệch) $w_0$(`w[0]`), nên với trọng số này cập nhật đơn giản là:
<!-- ```w[0] = ro[i]``` -->
$w_0 = \rho_i$

## Tác động của L1 penalty

Xét mô hình đơn giản có 2 đặc trưng:

In [64]:
simple_features = ['sqft_living', 'bedrooms']
my_output = 'price'
(simple_feature_matrix, output) = get_numpy_data(full_data, simple_features, my_output)

Đừng quên chuẩn hóa các đặc trưng:

In [65]:
simple_feature_matrix, norms = normalize_features(simple_feature_matrix)

Chúng ta gán một số tập hợp các trọng số ban đầu ngẫu nhiên và kiểm tra các giá trị của `ro[i]`:

In [66]:
weights = np.array([1., 4., 1.])

Sử dụng `predict_output()` để đưa ra dự đoán trên dữ liệu này. 

In [67]:
prediction = predict_output(simple_feature_matrix, weights)# easy
prediction

array([0.02675867, 0.04339256, 0.01990703, ..., 0.02289873, 0.03178473,
       0.02289873])

Tính giá trị của `ro[i]` cho từng đặc trưng trong mô hình đơn giản này sử dụng công thức đã cho:
<!-- ```ro[i] = SUM[ [feature_i]*(output - prediction + w[i]*[feature_i]) ]. ``` -->
$\rho_i = \sum feature_i * (output - prediction + w_i*feature_i)$

*Gợi ý: có thể sử dụng vectơ Numpy cho feature_i bằng:*
```
simple_feature_matrix[:,i]
```

In [68]:
# Nếu còn băn khoăn, hãy áp dụng trực tiếp công thức trên. 
ro = []
for i in range(len(weights)):
    ro_i = np.sum(simple_feature_matrix[:, i]*(output - prediction + weights[i]*simple_feature_matrix[:, i]))
    ro.append(ro_i)
print(f'ro values: {ro}')

ro values: [79400300.0145229, 87939470.82325175, 80966698.66623947]


***QUIZ***

Nhớ lại rằng, bất cứ khi nào `ro[i]` nằm trong khoảng `-l1_penalty/2` và `l1_penalty/2` thì trọng số `w[i]` tương ứng sẽ về 0. Bây giờ, giả sử chúng ta thực hiện một bước coordinate descent ở đặc trưng 1 hoặc đặc trưng 2. Phạm vi giá trị nào của `l1_penalty`sẽ **không đặt** `w[1]` thành 0 mà **đặt** `w[2]` thành 0 nếu chúng ta lấy một bước trong tọa độ đó?

In [69]:
# Return True if value is within the threshold ranges otherwise False
# Looking for range -l1_penalty/2 <= ro <= l1_penalty/2
def in_l1range(value, penalty):
    return (value >= -penalty/2.) and (value <= penalty/2.) 

***QUIZ***

Phạm vi giá trị nào của `l1_penalty` sẽ đặt **cả** `w[1]` và `w[2]` thành 0 nếu lấy một bước trong tọa độ đó?

Có thể nói rằng `ro[i]` định lượng tầm quan trọng của đặc trưng thứ i: `ro[i]` càng lớn thì đặc trưng thứ i càng có khả năng được giữ lại.

## Single Coordinate Descent Step

Using the formula above, implement coordinate descent that minimizes the cost function over a single feature i. Note that the intercept (weight 0) is not regularized. The function should accept feature matrix, output, current weights, l1 penalty, and index of feature to optimize over. The function should return new weight for feature i.

In [70]:
def lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty):
    # compute prediction
    prediction = predict_output(feature_matrix, weights)
    # compute ro[i] = SUM[ [feature_i]*(output - prediction + weight[i]*[feature_i]) ]
    ro_i = np.sum(feature_matrix[:, i]*(output - prediction + weights[i]*feature_matrix[:, i]))

    if i == 0: # intercept -- do not regularize
        new_weight_i = ro_i 
    elif ro_i < -l1_penalty/2.:
        new_weight_i = ro_i + l1_penalty/2
    elif ro_i > l1_penalty/2.:
        new_weight_i = ro_i - l1_penalty/2
    else:
        new_weight_i = 0.
    
    return new_weight_i

To test the function, run the following cell:

In [71]:
# should print 0.425558846691
import math
print(lasso_coordinate_descent_step(1, np.array([[3./math.sqrt(13),1./math.sqrt(10)],[2./math.sqrt(13),3./math.sqrt(10)]]), 
                                   np.array([1., 1.]), np.array([1., 4.]), 0.1) )

0.4255588466910251


## Cyclical coordinate descent 

Now that we have a function that optimizes the cost function over a single coordinate, let us implement cyclical coordinate descent where we optimize coordinates 0, 1, ..., (d-1) in order and repeat.

When do we know to stop? Each time we scan all the coordinates (features) once, we measure the change in weight for each coordinate. If no coordinate changes by more than a specified threshold, we stop.

For each iteration:
1. As you loop over features in order and perform coordinate descent, measure how much each coordinate changes.
2. After the loop, if the maximum change across all coordinates is falls below the tolerance, stop. Otherwise, go back to step 1.

Return weights

**IMPORTANT: when computing a new weight for coordinate i, make sure to incorporate the new weights for coordinates 0, 1, ..., i-1. One good way is to update your weights variable in-place. See following pseudocode for illustration.**
```
for i in range(len(weights)):
    old_weights_i = weights[i] # remember old value of weight[i], as it will be overwritten
    # the following line uses new values for weight[0], weight[1], ..., weight[i-1]
    #     and old values for weight[i], ..., weight[d-1]
    weights[i] = lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty)
    
    # use old_weights_i to compute change in coordinate
    ...
```

In [89]:
def lasso_cyclical_coordinate_descent(feature_matrix, output, initial_weights, l1_penalty, tolerance):
    D = feature_matrix.shape[1]
    weights = np.array(initial_weights)
    
    change = np.array(initial_weights) * 0.0
    converged = False
    
    while not converged:
        for i in range(D):
            old_weight = weights[i]
            weights[i] = lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty)

            change[i] = np.abs(weights[i] - old_weight)

        if max(change) < tolerance:
            converged = True
    return weights

Using the following parameters, learn the weights on the sales dataset. 

In [90]:
simple_features = ['sqft_living', 'bedrooms']
my_output = 'price'
initial_weights = np.zeros(3)
l1_penalty = 1e7
tolerance = 1.0

First create a normalized version of the feature matrix, `normalized_simple_feature_matrix`.

In [91]:
(simple_feature_matrix, output) = get_numpy_data(full_data, simple_features, my_output)
(normalized_simple_feature_matrix, simple_norms) = normalize_features(simple_feature_matrix) # normalize features

Then, run your implementation of LASSO coordinate descent:

In [92]:
weights = lasso_cyclical_coordinate_descent(normalized_simple_feature_matrix, output,
                                            initial_weights, l1_penalty, tolerance)
prediction = predict_output(normalized_simple_feature_matrix, weights)
rss = np.sum((output - prediction) ** 2)
print("{:.2E}".format(rss))
print(np.array(['constant'] + simple_features)[weights != 0])
# Result should be:
# # 1.63E+15
# # ['constant' 'sqft_living']

1.63E+15
['constant' 'sqft_living']


***QUIZ QUESTIONS***
1. What is the RSS of the learned model on the normalized dataset? (Hint: use the normalized feature matrix when you make predictions.)
2. Which features had weight zero at convergence?

# Evaluating LASSO fit with more features

Let us split the sales dataset into training and test sets.

In [93]:
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(full_data, train_size=0.8, test_size=0.2, random_state=0)

Let us consider the following set of features.

In [94]:
all_features = ['bedrooms',
                'bathrooms',
                'sqft_living',
                'sqft_lot',
                'floors',
                'waterfront', 
                'view', 
                'condition', 
                'grade',
                'sqft_above',
                'sqft_basement',
                'yr_built', 
                'yr_renovated']

First, create a normalized feature matrix from the TRAINING data with these features.  (Make you store the norms for the normalization, since we'll use them later)

In [95]:
# use normalize_features, goldfish.
my_output = 'price'
(feature_matrix, output) = get_numpy_data(train_data, all_features, my_output)
normalized_feature_matrix, norms = normalize_features(feature_matrix)

First, learn the weights with `l1_penalty=1e7`, on the training data. Initialize weights to all zeros, and set the `tolerance=1`.  Call resulting weights `weights1e7`, you will need them later.

In [96]:
# Handholding level: nonexistent. Not that you will need any at this point.
initial_weights = np.zeros(len(all_features) + 1)
l1_penalty = 1e7
tolerance = 1.0

In [98]:
weights1e7 = lasso_cyclical_coordinate_descent(normalized_feature_matrix, output,
                                               initial_weights, l1_penalty, tolerance)
weights1e7 

array([24625317.01587637,        0.        ,        0.        ,
       48292264.53592271,        0.        ,        0.        ,
        3566000.33284847,  7734349.09716174,        0.        ,
              0.        ,        0.        ,        0.        ,
              0.        ,        0.        ])

***QUIZ QUESTION***

What features had non-zero weight in this case?

In [99]:
feature_list = ['constant'] + all_features
print(feature_list)

['constant', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade', 'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated']


In [100]:
feature_weights1e7 = dict(zip(feature_list, weights1e7))
for k,v in feature_weights1e7.items():
    if v != 0.0:
        print(k, v)

constant 24625317.015876368
sqft_living 48292264.535922706
waterfront 3566000.3328484744
view 7734349.097161738


Next, learn the weights with `l1_penalty=1e8`, on the training data. Initialize weights to all zeros, and set the `tolerance=1`.  Call resulting weights `weights1e8`, you will need them later.

In [101]:
# Just change the number and try again.
l1_penalty=1e8
tolerance = 1.0
weights1e8 = lasso_cyclical_coordinate_descent(normalized_feature_matrix, output,
                                               initial_weights, l1_penalty, tolerance)
weights1e8

array([71373534.79051217,        0.        ,        0.        ,
              0.        ,        0.        ,        0.        ,
              0.        ,        0.        ,        0.        ,
              0.        ,        0.        ,        0.        ,
              0.        ,        0.        ])

***QUIZ QUESTION***

What features had non-zero weight in this case?

In [102]:
feature_weights1e8 = dict(zip(feature_list, weights1e8))
for k,v in feature_weights1e8.items():
    if v != 0.0:
        print(k, v)

constant 71373534.79051217


Finally, learn the weights with `l1_penalty=1e4`, on the training data. Initialize weights to all zeros, and set the `tolerance=5e5`.  Call resulting weights `weights1e4`, you will need them later.  (This case will take [quite a bit longer](https://xkcd.com/303/) to converge than the others above.)

In [103]:
# This is closer to your future than you might think. Unless you fail the course, but hey.
l1_penalty=1e4
tolerance=5e5
weights1e4 = lasso_cyclical_coordinate_descent(normalized_feature_matrix, output,
                                               initial_weights, l1_penalty, tolerance)
weights1e4

array([ 77664061.64562126, -19472165.15076051,  14984932.17111344,
        88451858.26250325,  -2367122.75899844,  -7580888.92952496,
         6722334.51845239,   7489670.90064896,   3978526.55485284,
        13776609.14004955, -11474194.80073022,  -4874702.39717918,
       -82543644.30150893,   2943815.78795278])

***QUIZ QUESTION***

What features had non-zero weight in this case?

In [104]:
feature_weights1e4 = dict(zip(feature_list, weights1e4))
for k,v in feature_weights1e4.items():
    if v != 0.0:
        print(k, v)

constant 77664061.64562126
bedrooms -19472165.15076051
bathrooms 14984932.171113444
sqft_living 88451858.26250325
sqft_lot -2367122.7589984355
floors -7580888.92952496
waterfront 6722334.518452395
view 7489670.900648957
condition 3978526.5548528424
grade 13776609.14004955
sqft_above -11474194.800730217
sqft_basement -4874702.397179184
yr_built -82543644.30150893
yr_renovated 2943815.7879527835


## Rescaling learned weights

Recall that we normalized our feature matrix, before learning the weights.  To use these weights on a test set, we must normalize the test data in the same way.

Alternatively, we can rescale the learned weights to include the normalization, so we never have to worry about normalizing the test data: 

In this case, we must scale the resulting weights so that we can make predictions with *original* features:
 1. Store the norms of the original features to a vector called `norms`:
```
features, norms = normalize_features(features)
```
 2. Run Lasso on the normalized features and obtain a `weights` vector
 3. Compute the weights for the original features by performing element-wise division, i.e.
```
weights_normalized = weights / norms
```
Now, we can apply `weights_normalized` to the test data, without normalizing it!

Create a normalized version of each of the weights learned above. (`weights1e4`, `weights1e7`, `weights1e8`).

In [106]:
# You remembered to store the weights, didn't you?
my_output = 'price'
(feature_matrix, output) = get_numpy_data(train_data, all_features, my_output)
normalized_feature_matrix, norms = normalize_features(feature_matrix)

In [107]:
normalized_weights1e7 = weights1e7 / norms
normalized_weights1e8 = weights1e8 / norms
normalized_weights1e4 = weights1e4 / norms
print("{:.4f}".format(normalized_weights1e7[3]))

161.2952


To check your results, if you call `normalized_weights1e7` the normalized version of `weights1e7`, then:
```
print("{:.4f}".format(normalized_weights1e7[3]))
```
should return 161.2952

## Evaluating each of the learned models on the test data

Let's now evaluate the three models on the test data:

In [108]:
(test_feature_matrix, test_output) = get_numpy_data(test_data, all_features, 'price')

Compute the RSS of each of the three normalized weights on the (unnormalized) `test_feature_matrix`:

In [109]:
# If you need more cells, Insert -> Insert Cell Below.
prediction =  predict_output(test_feature_matrix, normalized_weights1e7)
RSS = np.dot(test_output-prediction, test_output-prediction)
print('RSS for model with weights1e7 = ', RSS)

RSS for model with weights1e7 =  271730500343684.94


In [111]:
# If you need more cells, Insert -> Insert Cell Below.
prediction =  predict_output(test_feature_matrix, normalized_weights1e8)
RSS = np.dot(test_output-prediction, test_output-prediction)
print('RSS for model with weights1e8 = ', RSS)

RSS for model with weights1e8 =  514904176489231.25


In [112]:
prediction =  predict_output(test_feature_matrix, normalized_weights1e4)
RSS = np.dot(test_output-prediction, test_output-prediction)
print('RSS for model with weights1e4 = ', RSS)

RSS for model with weights1e4 =  225312368588044.56


***QUIZ QUESTION***

Which model performed best on the test data?<br>
==>model with weights1e4

================================END===========================